# Normality index

## Connect to the database

In [38]:
from databaseconnection import DatabaseConnection
gds = DatabaseConnection().get_database_connection()
gds.version()

'2.3.2'

## Create property `answersForAlternative` in `QuestionAlternative`



In [34]:

gds.run_cypher("""
                MATCH (a:Answer)-[chose:CHOSE]->(qa:QuestionAlternative)
                WITH qa as alternative, COUNT(chose) as answersForAlternative
                SET alternative.answersForAlternative = answersForAlternative
                RETURN alternative.name, alternative.answersForAlternative
               """)


,alternative.name,alternative.answersForAlternative
0,Årskurs 7,24
1,Tjej,29
2,Annat alternativ,1
3,Osäker på min sexuella läggning,2
4,Årskurs 8,20
...,...,...
1320,Saknas eller är inte aktuell,2
1321,Finns och är framtagen och reviderad årligen a...,37
1322,Finns och är framtagen och reviderad årligen i...,33
1323,Finns och är ’levande’ och aktuell i verksamhe...,12


## Create property `answersForQuestion` in `Question`

In [43]:
gds.run_cypher("""
               MATCH (q:Question)<-[iat:IS_ANSWER_TO]-(a:Answer)
               WITH q as question, COUNT(iat) as answersForQuestion
               SET question.answersForQuestion = answersForQuestion
               RETURN question.id, answersForQuestion
               ORDER BY answersForQuestion DESC
               """)

,question.id,answersForQuestion
0,6a0267b8-5220-4db3-98a6-a0d5bacc5190,2965
1,36393ae4-8a92-4c85-93b2-711f3759ba0b,2965
2,af247768-d27e-4e9b-94ac-a50e3a82e8c5,2965
3,b59bb433-0ca9-47f5-bf76-bfb97061612f,2965
4,4664611d-51f8-4f76-b398-3a0bfc988660,2965
...,...,...
285,fe32cb72-bec6-441b-8e21-aaf107792344,9
286,efd0ec8c-189e-4ab1-92fc-e647f0108436,9
287,4ac938fd-ec46-40ea-bef2-039cfc42156a,9
288,b9f669dd-f225-4ebb-96ab-aabf311e2b22,9


## Create property `normalityIndex` in `Respondent`

`normailtyIndex` is a property that is calculated from the sum of all the `Respondent`s `QuestionAlternative.answersForAlternative`/`Question.answersForQuestion`.

In [57]:

gds.run_cypher("""
               MATCH (r:Respondent)-[ha:HAS_ANSWERED]->(a:Answer)-[c:CHOSE]->(qa:QuestionAlternative)<-[co:CONSISTS_OF]-(q:Question)
               WITH qa.name as alternative, qa.answersForAlternative as answersForAlternative, q.id as question, q.answersForQuestion as answersForQuestion, id(r) as respondent
               WITH alternative, answersForAlternative, question, answersForQuestion, respondent, toFloat(answersForAlternative) / toFloat(answersForQuestion) as p_i
               WITH respondent, COUNT(p_i) as totalQuestions, SUM(p_i) as sum_p_i
               SET gds.util.asNode(respondent).normalityIndex = sum_p_i / totalQuestions
               RETURN respondent, sum_p_i / totalQuestions as normalityIndex
               ORDER BY normalityIndex DESC
               """)

,respondent,normalityIndex
0,16275,0.750422
1,107472,0.705287
2,189540,0.684265
3,127471,0.682476
4,173468,0.682476
...,...,...
3461,157966,0.065836
3462,105596,0.065836
3463,153944,0.065429
3464,198675,0.060663
